<a href="https://colab.research.google.com/github/jghcobussen/PRA3024/blob/master/Week_3_FinalVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import statements

!pip install sympy

from math import *
from sympy import *
from numpy import *
from scipy import *
from scipy.integrate import quad

In [0]:
# Define constants

G = 6.67408*(10**(-11))
c = 2.99792458*(10**8)

solarmass = 1.9891*(10**30)
m1 = 10*solarmass
m2 = 10*solarmass
Mtot = m1 + m2

f0 = 215
s0 = 10**(-49)
A = 10

value_E = (m1*m2)/Mtot
value_Mchirp = (((m1*m2)/Mtot)**(3/5))*Mtot
value_phic = 1
value_tc = 1


In [3]:
# Method to compute A using the function given in the exercise. However, the value for A turns out many powers higher than 10 and the reason for this is not clear.

def integrand_S(f, s0, f0):
  return (f**(-7/6))/(s0*((f/f0)**(-4.14)-5*(f/f0)**(-2)+111*(1-(f/f0)**2+((f/f0)**4)/2)/(1+((f/f0)**2)/2)))

def A_SNR(SNR, lowerbound, upperbound):
  Int_S = (quad(integrand_S, lowerbound, upperbound, args=(f0,s0)))
  S = 4*Int_S[0]
  A = sqrt(SNR/S)
  return A

print("The value of A is:")
A_SNR(10, 20, 300)

The value of A is:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: scipy.sqrt is deprecated and will be removed in SciPy 2.0.0, use numpy.lib.scimath.sqrt instead
  


9.242963185135387e+52

In [11]:
# Code that attempts to allow the user to add the desired PN to the function h. Variables are defined as 
# symbols using the sympy package.

# This code block is not finished yet!

f = Symbol('f')
phic = Symbol('phic')
tc = Symbol('tc')
E = Symbol('E')
Mchirp = Symbol('Mchirp')

v = (pi*((Mchirp/(E**3/5)))*f)**(1/3)
vlso = 1
gamma = 1
lamda = -0.6451
theta = -1.28

def PNvalue(pn):

  number = pn*2

  alpha0 = 1
  alpha1 = 0
  alpha2 = (20/9)*( (743/336) + (11/4)*E)
  alpha3 = -16*pi
  alpha4 = 10*((3058673/1016064) + (5429/1008)*E + (617/144)*(E**2))
  alpha5 = pi*( (38645/756) + (38645/252)*ln(((pi*(Mchirp/(E**(3/5)))*f)**(1/3))/vlso) - (65/9)*E*(1 + 3*(ln(((pi*(Mchirp/(E**(3/5)))*f)**(1/3))/vlso))))
  alpha6 = ( (11583231236531/4694215680) - ((640*(pi**2))/3) - (6848/21)*gamma) + E*((-15335597827/3048192) + (2255*(pi**2))/12 - (1760*theta)/3 + (12320*lamda)/9) + ((76055/1728)*(E**2) - (127825/1296)*(E**3) - (6848/21)*ln(4*((pi*(Mchirp/(E**(3/5)))*f)**(1/3))))
  alpha7 = pi*( (77096675/254016) + (378515/1512)*E - (74045/756)*(E**2))

  if number == 0:
    return alpha0
  elif number == 1:
    return alpha1
  elif number == 2:
    return alpha2
  elif number == 3:
    return alpha3
  elif number == 4:
    return alpha4
  elif number == 5:
    return alpha5
  elif number == 6:
    return alpha6
  elif number == 7:
    return alpha7
  else:
    return "PN out of bounds"

def PN_Summation(pn):
  
  summationvalue = 0
  
  for k in range (0, 2*pn-1):
    alpha = PNvalue(k)
    step = alpha*((pi*((Mchirp/(E**3/5)))*f)**(1/3))**(2*pn)
    summationvalue += step
  
  return summationvalue
    
PN_Summation(1)



6.27210400614069*(Mchirp*f/E**3)**0.666666666666667

In [0]:
# Code that defines the function of h in two ways: the first method simply returns h as a function of
# symbols; the second takes in values for the parameters and calculates the outcome. Additionally, a 
# list of the parameters of function h is defined for later use. 

varlist = [phic, tc, E, Mchirp]

def h_function(pn):
  h = A*(f**(-7/6))*(e**(i((2*pi*f*tc - phic - pi/4 + 3/(128*E*(((pi*(Mchirp/(E**3/5))*f)**(1/3))**5))))*(PN_Summation(pn))))
  return h

def h_function_value(x_phic, x_tc, x_E, x_Mchrip, x_f, pn):
  h = A*(f**(-(7/6)))*(2.71828**(i((2*(pi)*f*tc - phic - pi/4 + 3/(128*E*(((pi*(Mchirp/(E**(3/5)))*f)**(1/3))**5)))))*((PN_Summation(pn)))
  return h


In [0]:
# Definition to create empty list and convert it to a matrix.
# This was necessary while testing the integration later because the values were too big for a matrix and thus had to be put in a list first. 
# It may speak for itself that proper values in the list would not require this block of code and could be put directly into a matrix instead.

list = []

def list_to_matrix(list): 
  ar = array([[0, 0, 0, 0],
       [ 0, 0, 0, 0],
       [ 0, 0, 0, 0],
       [ 0, 0, 0, 0]])
  for j in range(0,3):
    for k in range(0,3):
      ar[j][k]= list[(j*4+k)]
  return ar

In [0]:
# Method to integrate a function of f numerically using the composite midpoint method. 

def integration(min, max, step, function):
  integral = 0
  for k in arange(min, (max-step), step):
    y1 = function.subs({f:k})
    y2 = function.subs({f:(k+step)})
    mean = (y1+y2)/2
    area = mean*step
    integral += area
  return integral

# Demonstration to show that the integration method works:

def y(f):
  y = 2*f+1
  return y

testfunction = y(f)
print("The testfunction looks like:")
print("")
print("y = " + str(testfunction))

integrated = integration(0, 10, 0.01, testfunction)
print("The value of the integral between the selected boundaries is:")
print("")
print(integrated)



In [0]:
# Define PSD of various detectors

def PSD_AdvancedLigo(f0, s0):
  psd = (s0*((f/f0)**(-4.14)-5*(f/f0)**(-2)+111*(1-(f/f0)**2+((f/f0)**4)/2)/(1+((f/f0)**2)/2)))
  return psd

In [0]:
# Loop that takes the 16 combinations of derivatives and computes the integral of each combination numerically, using the 
# integration method defined above, which uses the composite midpoint method, and then adds the values to a list. The differentiation
# is done using the sympy package. This requires all the variables of a function to be defined as symbols, with respect to 
# which the integration can be performed algebraically. After the integration the symbols are substituted with their acutal
# values and the result is calculated. 


# Ideally, the list_to_matrix method should be invoked to create a matrix, but due to radiculously high values this 
# gives an error message as those values are to large for a matrix. I have not found the source of the high results; 
# when testing the loop on simple functions it seemed to give accurate results. Possibly I have made a mistake in  
# rewriting the function in terms of Mchirp and Eta instead of Mtotal. I have tried to check this, but without any success. 

h = h_function()

for j in varlist:
  h1diff = h.diff(j)
  a = h1diff.subs({phic:value_phic, tc:value_tc, E:value_E, Mchirp:value_Mchirp})
  print("a")
  print(a)

  for k in varlist:
    h2diff = h.diff(k)
    b = h2diff.subs({phic:value_phic, tc:value_tc, E:value_E, Mchirp:value_Mchirp})
    print("b")
    print(b)

    y = (a*b)/(PSD_AdvancedLigo(215, (10**(-49))))
    print("y")
    print(y)

    subs = y.subs({f:100})
    print("subs")
    print(subs)

    integrated_value = integration(80.0, 120.0, 1, y)
    list.append(integrated_value)
print(list)

In [0]:
# Code to invert any matrix. Also computes the dot product of the matrix and its inverse to check whether 
# the result resembles the unit matrix. The input should be the matrix computed from the previous block,
# but since this is not functioning well this block computes a random 4x4 matrix to illustrate how the code
# works.

def create_4x4_matrix():
  m = array([[0, 0, 0, 0],
       [ 0, 0, 0, 0],
       [ 0, 0, 0, 0],
       [ 0, 0, 0, 0]])
  for k in range(0, 4):
    for j in range(0, 4):
      m[k][j] = random.randint(1,26)
  return m

def invert_matrix(matrix):
  print("The input matrix looks like this:")
  print(matrix)
  print("")

  inv_matrix = linalg.inv(matrix)
  print("The inverted matrix looks as follows:")
  print(inv_matrix)
  print("")

  unit_matrix = matrix.dot(inv_matrix)
  print("Check if the dot product is similar to the unit matrix:")
  print(unit_matrix)

matrix = create_4x4_matrix()
invert_matrix(matrix)